# EZ Model

Notebook to drive model discovery, data load, asking questions.

You discover model by chatting with an OpenAI LLM model. You visualize the model using PlantUML, a diagram-as-code tool. 

Once you're happy with the model, you create test data, again with help from OpenAI.

You then load that data into Amazon Neptune database.

Then you ask it questions. For this we use Langchain's Neptune OpenCypher integration with LLM. The question/answer flow is the following:
- During setup, the Langchain Neptune OpenCypher component introspects the Neptune database and extracts a graph schema.
- The end user asks as a natural language question.
- The LLM writes an OpenCypher query representing the intent of the question. It uses the graph schema extracted above.
- The Langchain Neptune OpenCypher component executes the query and returns back the result, represented as a JSON object.
- The LLM answers the end user's question in natural language based on the query result.

We also include a handy reverse engineer package; generate PlantUML diagram from data in the graph.

## Setup

### Notebook Pre-Req

You can upload this notebook into a Jupyter environment configured to use Neptune Workbench. I tested this on an Amazon Sagemaker notebook running Python 3.10.x. 

### Python Pre-Req

I tested this on an Amazon Sagemaker notebook running Python 3.10.x. You need Python 3.9 or higher.

### Neptune Pre-Req

Your Neptune cluster must run engine version 1.2.1.x.

### Setting key

You need an OpenAI API key with access to GPT-4 model. See https://platform.openai.com/account/billing/overview for instructions how to fund the key for use.

Once you have your key, add a file in the same folder as the notebook called *key.txt*. To do this: 
- From Jupyter, New | File
- Paste your key in the file
- Rename key.txt


In [ ]:
!pip install langchain

In [ ]:
!pip install openai

In [ ]:
!pip install iplantuml

In [ ]:
import iplantuml

In [ ]:
import os
import subprocess

GRAPH_NOTEBOOK_HOST= os.popen("source ~/.bashrc ; echo $GRAPH_NOTEBOOK_HOST").read().split("\n")[0]

# assumes your key is in key.txt in the same folder.
# if you prefer, uncomment the second line and hardcode the value 
# Decided not to use dotenv because saving as .env from Jupyter was not obvious. Doable from terminal, but wanted to dumb it down.
OPENAI_API_KEY = os.popen("cat key.txt | head -1").read().split("\n")[0]
#OPENAI_API_KEY ='<MYKEY>'

## Python version check


In [ ]:
%%bash 
# 3.9 or higher?
python --version

## Chat with LLM to find a basic model

In [ ]:
#x='Certainly! Here\'s the PlantUML class diagram representation of the knowledge graph for a company exclusion list:\n\n```\n@startuml\nclass Company {\n  - name: string\n  - industry: string\n  - exclusionReason: string\n}\n\nclass ExclusionList {\n  - companies: Company[]\n  + addCompany(company: Company): void\n  + getCompanies(): Company[]\n}\n\nclass KnowledgeGraph {\n  - exclusionList: ExclusionList\n  + addCompany(company: Company): void\n  + removeCompany(company: Company): void\n  + getExclusionReason(company: Company): string\n  + getExcludedCompanies(): Company[]\n}\n\nKnowledgeGraph "1" -- "1" ExclusionList\nExclusionList "1" -- "*" Company\n\n@enduml\n```\n\nIn this diagram, the `Company` class represents individual companies and has attributes like `name`, `industry`, and `exclusionReason`. The `ExclusionList` class contains a list of `Company` objects and provides methods for adding and retrieving companies. The `KnowledgeGraph` class represents the overall knowledge graph and has methods for managing companies and retrieving exclusion information.\n\nPlease note that this is just one possible representation and can be customized based on specific requirements and additional relationships between classes.'
#y='Sure! Here is an example of a PlantUML class diagram representing a knowledge graph for a company exclusion list:\n\n```puml\n@startuml\nclass KnowledgeGraph {\n  +nodes: List<Node>\n  +edges: List<Edge>\n}\n\nclass Node {\n  +id: String\n  +label: String\n  +properties: Map<String, Object>\n}\n\nclass Edge {\n  +id: String\n  +label: String\n  +sourceNodeId: String\n  +targetNodeId: String\n  +properties: Map<String, Object>\n}\n\nKnowledgeGraph "1" --> "n" Node\nKnowledgeGraph "1" --> "n" Edge\n@enduml\n```\n\nIn this diagram, the `KnowledgeGraph` class represents the overall knowledge graph, which consists of a list of `Node` objects and a list of `Edge` objects. Each `Node` represents a concept or entity in the knowledge graph and has an identifier (`id`), a label, and a set of properties. Similarly, each `Edge` represents a relationship between two nodes and has an identifier, a label, source and target node identifiers, and a set of properties.\n\nPlease note that this diagram is a simplified representation and may not capture all the specific details of your desired company exclusion list knowledge graph. Let me know if you have any specific requirements or if you would like any further modifications to this diagram!'

def extract_codeblocks(res):
    curr_block = None
    blocks = []
    toks = res.split("```")
    for t in toks:
        if curr_block is None:
            curr_block = {'text': "", 'type': ""}
        else:
            first_nl = t.find("\n")
            curr_block['type'] = t[0:first_nl]
            curr_block['text'] = t[first_nl:].strip()
            blocks.append(curr_block)
            curr_block = None
    return blocks

#extract_codeblocks(y)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain  
  
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.9)
conversation = ConversationChain(llm=chat)  

### Q1 - Show a knowledge graph model for a company exclusion list
<details><summary>Click to view/hide sample answer</summary>
<p>
Sure! One popular knowledge graph model that can be used for a company exclusion list is the Resource Description Framework (RDF) model. RDF provides a way to represent knowledge and information in a graph-like structure, where nodes represent entities and edges represent relationships between those entities.

To create a knowledge graph model for a company exclusion list, you could define entities such as "companies," "exclusion criteria," and "exclusion list." The relationships between these entities can be represented using RDF properties.

For example, you could have a company entity with properties like "name," "industry," and "location." The exclusion criteria entity could have properties like "reason," "severity," and "start/end dates." Finally, the exclusion list entity could have properties like "exclusion reason," "date added," and "responsible authority."

By linking these entities and properties together using RDF triples, you can create a knowledge graph model that represents the relationships between companies, exclusion criteria, and the exclusion list itself. This model can be used to store and query information about which companies are included in the exclusion list, the reasons for their exclusion, and other relevant details.
</p>
</details>

In [ ]:
res = conversation.run("Show a knowledge graph model for a company exclusion list")
print(res)

### Q2 - Show as PlantUML class diagram
<details><summary>Click to view/hide sample answer</summary>
<p>
Certainly! Here is a PlantUML class diagram representing the knowledge graph model for a company exclusion list:

```
@startuml
class Company {
  - name: string
  - industry: string
  - location: string
}

class ExclusionCriteria {
  - reason: string
  - severity: string
  - startEndDate: string
}

class ExclusionList {
  - exclusionReason: string
  - dateAdded: string
  - responsibleAuthority: string
}

Company -- "1" ExclusionCriteria
Company -- "0..*" ExclusionList
ExclusionList -- "1" ExclusionCriteria
@enduml
```

In this diagram, we have three classes representing entities in the knowledge graph model: "Company," "ExclusionCriteria," and "ExclusionList." Each class has its properties defined as private attributes.

The "Company" class has properties like "name," "industry," and "location." The "ExclusionCriteria" class has properties like "reason," "severity," and "startEndDate." The "ExclusionList" class has properties like "exclusionReason," "dateAdded," and "responsibleAuthority."

The relationships between these classes are represented by associations. A company can have one or more exclusion criteria, so there is a "1" to "*" association between "Company" and "ExclusionCriteria." Similarly, a company can be included in multiple exclusion lists, and an exclusion list can have multiple exclusion criteria. Therefore, there are associations between "Company" and "ExclusionList" as well as between "ExclusionList" and "ExclusionCriteria," both represented by a "1" to "*" association.

This PlantUML class diagram provides a visual representation of the knowledge graph model for a company exclusion list, showcasing the relationships between entities and their properties.
    
</p>
</details>

In [ ]:
res = conversation.run("Show as PlantUML class diagram")
uml = extract_codeblocks(res)
print(res)

### Show PlantUML diagram
<details><summary>Click to view/hide sample image</summary>
<p>
    <img src="https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/plantuml/plantuml1.png"/>   
</p>
</details>

In [ ]:
print(str(len(uml)))
ipython = get_ipython()
ipython.run_cell_magic("plantuml", "", uml[0]['text'])

### Q3 - Include how companies related to each other
<details><summary>Click to view/hide sample answer</summary>
<p>
Certainly! In the context of a knowledge graph model for a company exclusion list, the relationships between companies can be represented in various ways. Here are a few examples:

1. Parent-Subsidiary Relationship: Companies can have a parent company or subsidiaries. This relationship can be represented using an additional property in the "Company" class, such as "parentCompany" or "subsidaries." For example, you could add a property called "parentCompany" with a data type of "Company" in the "Company" class to represent the parent company of a particular company.

2. Mergers and Acquisitions: Companies can merge with or acquire other companies. This relationship can be represented using an association class or an additional property. For example, you could create an association class called "MergersAndAcquisitions" that has properties like "acquiringCompany," "acquiredCompany," and "date." This association class represents the relationship between two companies involved in a merger or acquisition.

3. Competitor Relationship: Companies can have competitors within the same industry. This relationship can be represented using an additional property in the "Company" class, such as "competitors." For example, you could add a property called "competitors" with a data type of "Company[]" in the "Company" class to represent the list of competitors for a particular company.

These are just a few examples of how companies can be related to each other within the context of a knowledge graph model for a company exclusion list. The specific relationships and their representations may vary depending on the requirements and the domain of the application.    
</p>
</details>

In [ ]:
res = conversation.run("Include how companies related to each other")
uml = extract_codeblocks(res)
print(res)

### Q4 - Show that as PlantUML
<details><summary>Click to view/hide sample answer</summary>
<p>
Certainly! Here is an updated PlantUML class diagram representing the relationships between companies in the context of a knowledge graph model for a company exclusion list:

```
@startuml
class Company {
  - name: string
  - industry: string
  - location: string
  - parentCompany: Company
  - subsidiaries: Company[]
  - competitors: Company[]
}

class ExclusionCriteria {
  - reason: string
  - severity: string
  - startEndDate: string
}

class ExclusionList {
  - exclusionReason: string
  - dateAdded: string
  - responsibleAuthority: string
}

class MergersAndAcquisitions {
  - acquiringCompany: Company
  - acquiredCompany: Company
  - date: string
}

Company -- "1" ExclusionCriteria
Company -- "0..*" ExclusionList
ExclusionList -- "1" ExclusionCriteria
Company -- "1" Company: parentCompany
Company -- "*" Company: subsidiaries
Company -- "*" Company: competitors
Company -- "*" MergersAndAcquisitions: acquiringCompany
Company -- "*" MergersAndAcquisitions: acquiredCompany
@enduml
```

In this updated diagram, the "Company" class now includes additional properties to represent the relationships between companies. The "parentCompany" property represents the parent company of a particular company, while the "subsidiaries" property represents a list of subsidiaries for a particular company. The "competitors" property represents a list of competitors for a particular company.

Additionally, I have added a new class called "MergersAndAcquisitions" that represents the relationship between two companies involved in a merger or acquisition. This class has properties like "acquiringCompany," "acquiredCompany," and "date."

The associations between these classes have also been updated to reflect the new relationships. There are associations between "Company" and "MergersAndAcquisitions" for both acquiring and acquired companies.

This updated PlantUML class diagram provides a more comprehensive representation of the relationships between companies in the knowledge graph model for a company exclusion list.
</p>
</details>

In [ ]:
res = conversation.run("Show that as PlantUML")
uml = extract_codeblocks(res)
print(res)

### Show PlantUML Diagram
<details><summary>Click to view/hide sample image</summary>
<p>
    <img src="https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/plantuml/plantuml2.png"/>   
</p>
</details>

In [ ]:
ipython = get_ipython()
ipython.run_cell_magic("plantuml", "", uml[0]['text'])

### Let's tweak a bit

In [ ]:
good_model_content = """
class Company {
    +Name: String
    +Industry: String
    +Location: String
}

class ExclusionCriteria {
    +Type: String
    +Description: String
    +SeverityLevel: String
}

class Exclusion {
    +ExclusionStartDate: Date
    +ExclusionEndDate: Date
    +ExclusionType: String
}

Company "1" --> "*" Exclusion : has exclusion 
Company "1" --> "0..1" Exclusion : has current exclusion 
Exclusion "1" --> "1" ExclusionCriteria : has criteria

Company "1" --> "*" Company: related to
note on link : reltype: String, established date: Date
"""

good_model= "@startuml\n" + good_model_content + "\n@enduml\n"



### Tweaked PlantUML
<details><summary>Click to view/hide image</summary>
<p>
    <img src="https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/plantuml/plantuml3.png"/>   
</p>
</details>

In [ ]:
ipython = get_ipython()
ipython.run_cell_magic("plantuml", "", good_model)

## Get some test data for our model

<details><summary>Click to view/hide sample answers</summary>
<p>
    
Here is what sample response for OpenCypher looks like:

```
CREATE (:Company {Name: 'Company A', Industry: 'Tech', Location: 'California'})
CREATE (:Company {Name: 'Company B', Industry: 'Finance', Location: 'New York'})
CREATE (:Company {Name: 'Company C', Industry: 'Retail', Location: 'Texas'})

CREATE (:ExclusionCriteria {Type: 'Financial', Description: 'Non-payment of invoices', SeverityLevel: 'High'})
CREATE (:ExclusionCriteria {Type: 'Legal', Description: 'Violation of labor laws', SeverityLevel: 'Medium'})
CREATE (:ExclusionCriteria {Type: 'Ethical', Description: 'Unethical business practices', SeverityLevel: 'Low'})

MATCH (companyA:Company {Name: 'Company A'})
MATCH (companyB:Company {Name: 'Company B'})
MATCH (financial:ExclusionCriteria {Type: 'Financial'})
CREATE (companyA)-[:has current exclusion]->(:Exclusion {ExclusionStartDate: date('2022-01-01'), ExclusionEndDate: date('2022-12-31'), ExclusionType: 'Financial'})
CREATE (companyA)-[:has exclusion]->(:Exclusion {ExclusionStartDate: date('2020-01-01'), ExclusionEndDate: null, ExclusionType: 'Legal'})
CREATE (companyA)-[:has exclusion]->(:Exclusion {ExclusionStartDate: date('2021-01-01'), ExclusionEndDate: date('2021-12-31'), ExclusionType: 'Ethical'})
CREATE (companyA)-[:has exclusion]->(:Exclusion {ExclusionStartDate: date('2020-01-01'), ExclusionEndDate: null, ExclusionType: 'Financial'})
CREATE (companyB)-[:has exclusion]->(:Exclusion {ExclusionStartDate: date('2022-01-01'), ExclusionEndDate: null, ExclusionType: 'Legal'})
CREATE (companyB)-[:has exclusion]->(:Exclusion {ExclusionStartDate: date('2021-01-01'), ExclusionEndDate: date('2021-12-31'), ExclusionType: 'Ethical'})
CREATE (companyB)-[:has exclusion]->(:Exclusion {ExclusionStartDate: date('2020-01-01'), ExclusionEndDate: null, ExclusionType: 'Financial'})
CREATE (companyA)-[:related to {reltype: 'Partner', established date: date('2020-01-01')}]->(companyB)
CREATE (companyA)-[:related to {reltype: 'Supplier', established date: date('2021-01-01')}]->(companyC)
```
    
And sample for RDF:

```
@prefix : <http://example.com/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:CompanyA rdf:type :Company ;
          :Name "Company A" ;
          :Industry "Tech" ;
          :Location "California" .

:CompanyB rdf:type :Company ;
          :Name "Company B" ;
          :Industry "Finance" ;
          :Location "New York" .

:CompanyC rdf:type :Company ;
          :Name "Company C" ;
          :Industry "Retail" ;
          :Location "Texas" .

:Financial rdf:type :ExclusionCriteria ;
           :Type "Financial" ;
           :Description "Non-payment of invoices" ;
           :SeverityLevel "High" .

:Legal rdf:type :ExclusionCriteria ;
       :Type "Legal" ;
       :Description "Violation of labor laws" ;
       :SeverityLevel "Medium" .

:Ethical rdf:type :ExclusionCriteria ;
         :Type "Ethical" ;
         :Description "Unethical business practices" ;
         :SeverityLevel "Low" .

:ExclusionA rdf:type :Exclusion ;
            :ExclusionStartDate "2022-01-01"^^xsd:date ;
            :ExclusionEndDate "2022-12-31"^^xsd:date ;
            :ExclusionType "Financial" ;
            :hasCriteria :Financial .

:ExclusionB rdf:type :Exclusion ;
            :ExclusionStartDate "2020-01-01"^^xsd:date ;
            :ExclusionType "Legal" ;
            :hasCriteria :Legal .

:ExclusionC rdf:type :Exclusion ;
            :ExclusionStartDate "2021-01-01"^^xsd:date ;
            :ExclusionEndDate "2021-12-31"^^xsd:date ;
            :ExclusionType "Ethical" ;
            :hasCriteria :Ethical .

:ExclusionD rdf:type :Exclusion ;
            :ExclusionStartDate "2020-01-01"^^xsd:date ;
            :ExclusionType "Financial" .

:ExclusionE rdf:type :Exclusion ;
            :ExclusionStartDate "2022-01-01"^^xsd:date ;
            :ExclusionType "Legal" .

:ExclusionF rdf:type :Exclusion ;
            :ExclusionStartDate "2021-01-01"^^xsd:date ;
            :ExclusionEndDate "2021-12-31"^^xsd:date ;
            :ExclusionType "Ethical" .

:CompanyA :hasCurrentExclusion :ExclusionA ;
          :hasExclusion :ExclusionB, :ExclusionC, :ExclusionD ;
          :relatedTo [ :reltype "Partner" ; :establishedDate "2020-01-01"^^xsd:date ] .

:CompanyB :hasExclusion :ExclusionE, :ExclusionF ;
          :relatedTo [ :reltype "Supplier" ; :establishedDate "2021-01-01"^^xsd:date ] .

:CompanyA :relatedTo :CompanyB .
:CompanyA :relatedTo :CompanyC .
```
    
</p>
</details>

In [ ]:
res = conversation.run("Generate OpenCypher queries to create sample data based on " + good_model_content)
#ocblock=extract_codeblocks(res)
print(res)

In [ ]:
res = conversation.run("Generate RDF sample data based on " + good_model_content)
#ocblock=extract_codeblocks(res)
print(res)

## Load the data from OC into Neptune

In [ ]:
%%oc

CREATE (:Company {`~id`: 'compA', Name: 'Company A', Industry: 'Tech', Location: 'California'})
CREATE (:Company {`~id`: 'compB', Name: 'Company B', Industry: 'Finance', Location: 'New York'})
CREATE (:Company {`~id`: 'compC', Name: 'Company C', Industry: 'Retail', Location: 'Texas'})
CREATE (:Company {`~id`: 'compD', Name: 'Company D', Industry: 'Retail', Location: 'Texas'})
CREATE (:Company {`~id`: 'compE', Name: 'Company E', Industry: 'Retail', Location: 'Texas'})

CREATE (:ExclusionCriteria {`~id`: 'xfin', Type: 'Financial', Description: 'Non-payment of invoices', SeverityLevel: 'High'})
CREATE (:ExclusionCriteria {`~id`: 'xleg', Type: 'Legal', Description: 'Violation of labor laws', SeverityLevel: 'Medium'})
CREATE (:ExclusionCriteria {`~id`: 'xeth',Type: 'Ethical', Description: 'Unethical business practices', SeverityLevel: 'Low'})

CREATE (:Exclusion {`~id`: 'xa2', ExclusionStartDate: datetime('2022-01-01T00:00:01'), ExclusionType: 'Financial'})
CREATE (:Exclusion {`~id`: 'xa1', ExclusionStartDate: datetime('2020-01-01T00:00:01'), ExclusionEndDate: datetime('2020-06-11T00:00:01'), ExclusionType: 'Financial'})
CREATE (:Exclusion {`~id`: 'xb1', ExclusionStartDate: datetime('1985-01-01T00:00:01'), ExclusionType: 'Legal'})
CREATE (:Exclusion {`~id`: 'xc1', ExclusionStartDate: datetime('1985-01-01T00:00:01'), ExclusionStartDate: datetime('1989-01-01T00:00:01'), ExclusionType: 'Legal'})
                                 

In [ ]:
%%oc

MATCH (companyA:Company {Name: 'Company A'}),                   
      (companyB:Company {Name: 'Company B'}), 
      (companyC:Company {Name: 'Company C'}),
      (companyD:Company {Name: 'Company D'}),
      (xa2:Exclusion {`~id`: 'xa2'}),
      (xa1:Exclusion {`~id`: 'xa1'}),
      (xb1:Exclusion {`~id`: 'xb1'}),
      (xc1:Exclusion {`~id`: 'xc1'})
CREATE (companyA)-[r1:relatedTo {reltype: 'Partner', establishedDate: datetime('2020-01-01T00:00:01')}]->(companyB)
CREATE (companyA)-[r2:relatedTo {reltype: 'Supplier', establishedDate: datetime('2021-01-01T00:00:01')}]->(companyC)
CREATE (companyC)-[r3:relatedTo {reltype: 'Supplier', establishedDate: datetime('2021-01-01T00:00:01')}]->(companyD)
CREATE (companyA)-[:hasCurrentExclusion]->(xa2)
CREATE (companyA)-[:hasExclusion]->(xa2)
CREATE (companyA)-[:hasExclusion]->(xa1)
CREATE (companyB)-[:hasCurrentExclusion]->(xb1)
CREATE (companyB)-[:hasExclusion]->(xb1)
CREATE (companyC)-[:hasExclusion]->(xc1)

RETURN r1, r2

## Create Neptune Langchain integration
Neptune 1.2.1.x required

In [ ]:
from langchain.graphs import NeptuneGraph
import os

host=GRAPH_NOTEBOOK_HOST
port = 8182

graph = NeptuneGraph(host=host, port=port)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import NeptuneOpenCypherQAChain

model='gpt-4'

llm = ChatOpenAI(temperature=0, model_name=model, openai_api_key=OPENAI_API_KEY)

chain = NeptuneOpenCypherQAChain.from_llm(
    llm=llm, graph=graph, verbose=True, top_K=10, return_intermediate_steps=True, return_direct=False)

## Ask questions

### Q1 - Which companies have financial exclusions

<details><summary>Click to view/hide sample answer</summary>
<p>
    
Generated Cypher:
    
MATCH (c:Company)-[:hasExclusion]->(e:Exclusion) 
WHERE e.ExclusionType = 'Financial' 
RETURN c.Name
    
Full Context:
{'results': [{'c.Name': 'Company A'}, {'c.Name': 'Company A'}]}

'The company that has financial exclusions is Company A.'
    
</p>
</details>

In [ ]:
chain.run('''Which companies have financial exclusions''')

### Q2 - Which companies have current exclusions

<details><summary>Click to view/hide sample answer</summary>
<p>
    
Generated Cypher:
    
MATCH (c:Company)-[:hasExclusion]->(e:Exclusion) 
WHERE e.ExclusionType = 'Financial' 
RETURN c.Name
    
Full Context:
{'results': [{'c.Name': 'Company A'}, {'c.Name': 'Company A'}]}

'The companies that currently have exclusions are Company A and Company B.'    
</p>
</details>

In [ ]:
chain.run('''Which companies have current exclusions''')

### Q3 - Which companies do not have a current exclusion
<details><summary>Click to view/hide sample answer</summary>
<p>
    
Generated Cypher:
    
MATCH (c:Company)
WHERE NOT (c)-[:hasCurrentExclusion]->(:Exclusion)
RETURN c.Name    

Full Context:
{'results': [{'c.Name': 'Company C'}, {'c.Name': 'Company D'}, {'c.Name': 'Company E'}]}

'The companies that do not have a current exclusion are Company C, Company D, and Company E.'
</p>
</details>

In [ ]:
chain.run('''Which companies do not have a current exclusion''')

### Q4 - Which companies located in Texas have an exclusion or are directly related to a company with an exclusion
<details><summary>Click to view/hide sample answer</summary>
<p>
    
Generated Cypher:
    
MATCH (c:Company)-[:hasExclusion|:hasCurrentExclusion]->(:Exclusion)
WHERE c.Location = 'Texas'
RETURN c.Name AS Company

UNION

MATCH (c:Company)-[:relatedTo]->(related:Company)-[:hasExclusion|:hasCurrentExclusion]->(:Exclusion)
WHERE c.Location = 'Texas'
RETURN related.Name AS Company

Full Context:
{'results': [{'Company': 'Company C'}]}

'The company located in Texas that has an exclusion or is directly related to a company with an exclusion is Company C.'
</p>
</details>

In [ ]:
chain.run('''Which companies located in Texas have an exclusion or are directly related to a company with an exclusion''')

### Q5 - Which companies are related directly to Company A
<details><summary>Click to view/hide sample answer</summary>
<p>
    
Generated Cypher:
    
MATCH (c1:Company {Name: 'Company A'})-[:relatedTo]->(c2:Company)
RETURN c2.Name

Full Context:
{'results': [{'c2.Name': 'Company C'}, {'c2.Name': 'Company B'}]}

'The companies that are directly related to Company A are Company C and Company B.'
</p>
</details>

In [ ]:
chain.run('''Which companies are related directly to Company A''')

### Q6 - Which companies are related directly or indirectly to Company A
<details><summary>Click to view/hide sample answer</summary>
<p>
    
Generated Cypher:
    
MATCH path = (c:Company {Name: 'Company A'})-[:relatedTo*]-(relatedCompany:Company)
RETURN DISTINCT relatedCompany.Name

Full Context:
{'results': [{'relatedCompany.Name': 'Company D'}, {'relatedCompany.Name': 'Company B'}, {'relatedCompany.Name': 'Company C'}]}

'The companies that are directly or indirectly related to Company A are Company D, Company B, and Company C.'
</p>
</details>

In [ ]:
chain.run('''Which companies are related directly or indirectly to Company A''')